<a href="https://colab.research.google.com/github/JamieBali/sudoku/blob/main/CNN_Sudoku_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import os, time

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data

In [ ]:
filepath = "/content/gdrive/My Drive/ColabNotebooks/IP/sudoku.csv"
inp = pd.read_csv(filepath)

puzzles = inp["quizzes"]
solutions = inp["solutions"]

In [ ]:
trivial = [[8,9,1,2,7,0,5,6,3],[6,0,3,1,8,5,9,0,0],[4,5,7,6,3,9,0,0,2],[5,0,6,4,1,7,2,3,0],[7,0,2,9,0,3,8,1,6],[3,1,0,0,2,6,0,5,4],[9,3,8,5,4,0,6,7,0],[1,6,4,0,9,0,3,2,5],[0,7,5,3,6,1,4,9,8]]

# this is a very simple 4x4 sudoku from sudoku.com
four = [[0,3,4,0],[4,0,0,2],[1,0,0,3],[0,2,1,0]]

# these three were taken randomly from sudoku.com
easy = [[0,0,0,0,7,9,0,3,0],[5,0,2,0,6,1,4,7,8],[3,7,6,0,8,5,9,0,2],[0,1,7,5,0,0,8,0,0],[2,0,9,8,3,0,0,0,0],[0,0,0,0,2,0,0,4,0],[0,0,0,0,5,0,2,0,1],[0,2,3,0,0,0,0,5,4],[1,0,0,7,0,0,0,0,0]]
medium = [[0,3,1,0,5,0,0,2,0],[0,0,0,0,0,2,9,0,5],[2,0,0,0,1,0,0,0,0],[3,5,0,0,9,0,0,7,0],[7,0,0,5,0,0,0,4,0],[0,1,0,7,0,3,2,0,0],[1,2,6,3,0,0,0,0,0],[0,9,0,8,0,5,0,0,0],[5,0,0,0,2,0,7,0,0]]
hard = [[0,4,0,0,0,5,0,6,0],[0,0,5,4,2,0,0,0,0],[0,0,1,6,0,3,5,0,4],[0,0,0,0,0,0,7,0,0],[0,3,7,0,0,0,0,1,0],[9,0,0,0,0,4,3,5,0],[0,0,4,2,5,0,0,0,0],[0,0,0,0,0,0,0,7,6],[6,0,9,0,7,0,0,0,5]]

# this is the solved version of the "easy" sudoku above.
solved = [[8,4,1,2,7,9,6,3,5],[5,9,2,3,6,1,4,7,8],[3,7,6,4,8,5,9,1,2],[4,1,7,5,9,6,8,2,3],[2,5,9,8,3,4,1,6,7],[6,3,8,1,2,7,5,4,9],[7,6,4,9,5,3,2,8,1],[9,2,3,6,1,8,7,5,4],[1,8,5,7,4,2,3,9,6]]

# Model Creation and Training

In [ ]:
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape
from sklearn.model_selection import train_test_split

class sudokuSolver():

  model = None

  def __init__(self, pre_trained = 0):
    if pre_trained == 1:
      self.model = keras.models.load_model("/content/gdrive/My Drive/ColabNotebooks/IP/model.keras")
    else:
      self.model = keras.models.Sequential()                            # create sequential model

      self.model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1))) # these layers come from a towards data science article
      self.model.add(BatchNormalization())                                                                  # see our paper for the reference.
      self.model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
      self.model.add(BatchNormalization())
      self.model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

      self.model.add(Flatten())
      self.model.add(Dense(81*9))               # convert to probabilities
      self.model.add(Reshape((-1, 9)))          # shape so each val has 9 probabilities
      self.model.add(Activation('softmax'))     # and find the highest probability

      self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  
  def format_puzzle(self, puzzle, from_dataset = False):
    if from_dataset:                                # dataset puzzles are formatted differently to ones we have written in manually
      formatted_puzzle = []
      for x in puzzle:
        formatted_puzzle.append(int(x))
      return self.normalise(np.array(formatted_puzzle)).reshape((9,9,1))
    else:
      formatted_puzzle = []
      for x in range(9):
        row = []
        for y in puzzle[x]:
          row.append(int(y))
        formatted_puzzle.append(row)
      formatted_puzzle = np.array(formatted_puzzle)
      return self.normalise(formatted_puzzle).reshape((1,9,9,1))

  def normalise(self, val):       # we want to our data to be zero centred when it goes into the network
    return (val/9) - 0.5

  def save_model(self, name):             # this allows us to save our mdeol so we don't have to repeatedly re-train it
    self.model.save("/content/gdrive/My Drive/ColabNotebooks/IP/" + name)

  def train(self, tr_puzzles, tr_solutions, split = 0.2): # format our puzzles, split the data, and then train the model
    train_puzzles = []
    train_solutions = []
    for x in tr_puzzles:
      train_puzzles.append(self.format_puzzle(x, True))         # format the puzzles and solutions
    for x in tr_solutions:
      train_solutions.append(np.array([int(t) for t in x]).reshape((81,1))-1)

    train_puzzles = np.array(train_puzzles)                     # convert to np array
    train_solutions = np.array(train_solutions)

    train_puzzles, test_puzzles, train_solutions, test_solutions = train_test_split(tr_puzzles, tr_solutions, test_size=split) # split the data

    print("Training with " + str(len(train_puzzles)) + " puzzles.")

    self.model.fit(train_puzzles, train_solutions, batch_size = 64, epochs=5, verbose=1) # inbuilt training function. 

  def get_model(self):  # returns a copy of the model
    return self.model

  def solve(self, puzzle):                      
    solution = np.copy(puzzle).reshape((9,9))           # create a copy of the output state
    while True:                                                   
      formatted_puzzle = self.format_puzzle(solution)   # format the current output state

      output = self.model.predict(formatted_puzzle).squeeze() # predict output

      predictions = np.argmax(output, axis = 1).reshape((9,9)) + 1                # find the highest probability output for each tile
      probabilities = np.around(np.max(output, axis=1).reshape((9,9)), 2)     

      mask = np.array(solution == 0)  # create a mask, so that we don't update givens
      if mask.sum() == 0:
        break

      probabilities *= mask         # multiply probabilities by the mask, removing the probability of givens

      index_of_highest = np.argmax(probabilities)               # and update the board state to contain the value with the highest probability
      x, y = (index_of_highest // 9), (index_of_highest % 9)

      solution[x][y] = predictions[x][y]

    return solution         # repeat until solution is found

# Training the Solver

In [ ]:
model = sudokuSolver()
model.train(puzzles[:100], solutions[:100], 0.99)


Epoch 1/10
7813/7813 [==============================] - 85s 11ms/step - loss: 0.5390
Epoch 2/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3700
Epoch 3/10
7813/7813 [==============================] - 85s 11ms/step - loss: 0.3522
Epoch 4/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3376
Epoch 5/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3271
Epoch 6/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3195
Epoch 7/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3139
Epoch 8/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3096
Epoch 9/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3061
Epoch 10/10
7813/7813 [==============================] - 84s 11ms/step - loss: 0.3032


In [ ]:
model.solve(hard)

array([[7, 4, 2, 8, 1, 5, 9, 6, 3],
       [3, 6, 5, 4, 2, 7, 1, 8, 7],
       [8, 9, 1, 6, 9, 3, 5, 2, 4],
       [5, 2, 8, 1, 3, 9, 7, 4, 2],
       [4, 3, 7, 5, 6, 2, 9, 1, 8],
       [9, 1, 6, 7, 8, 4, 3, 5, 2],
       [1, 7, 4, 2, 5, 6, 8, 3, 9],
       [2, 5, 3, 9, 8, 8, 4, 7, 6],
       [6, 8, 9, 3, 7, 1, 2, 9, 5]])

# Solver

In [ ]:
game = medium
with tf.device("/device:GPU:0"):
  model = sudokuSolver(6)
  print(model.solve(game))

[[9 3 1 6 5 8 4 2 7]
 [6 7 8 4 3 2 9 1 5]
 [2 4 5 9 1 7 6 8 3]
 [3 5 4 2 9 6 8 7 1]
 [7 6 2 5 8 1 3 4 9]
 [8 1 9 7 4 3 2 5 6]
 [1 2 6 3 7 4 5 9 8]
 [4 9 7 8 6 5 1 3 2]
 [5 8 3 1 2 9 7 6 4]]
